In [14]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [15]:
df = pd.read_excel("Updated_Data_Historis_2015_2025.xlsx", parse_dates=['Tanggal']) #parse = untuk ngubah kolom Tanggal jadi datetime
df = df.sort_values('Tanggal')
df = df.reset_index(drop=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4020 non-null   datetime64[ns]
 1   Temperatur Minimum                  4020 non-null   object        
 2   Temperatur Maksimum                 4020 non-null   object        
 3   Temperatur Rata-rata                4020 non-null   object        
 4   Kelembapan Rata-rata                4020 non-null   object        
 5   Curah Hujan (mm)                    4020 non-null   object        
 6   Lamanya Penyinaran Matahari         4020 non-null   object        
 7   Kecepatan Angin Maksimum            4020 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4020 non-null   int64         
 9   Kecepatan Angin Rata-rata           4020 non-null   int64         
 10  Arah Angin Terbanyak (°)

In [17]:
checkstripvalue = (df['Curah Hujan (mm)'] == "-").sum()
print(checkstripvalue)

497


In [18]:
df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].replace('-',0)

df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].astype(float)

C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\2154469297.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].replace('-',0)


In [19]:
checkstripvalue2 = (df['Curah Hujan (mm)'] == 8888).sum()
print(checkstripvalue2)

209


In [20]:
mapping = {
    'C': 0, #Calm
    'N': 0, #North
    'NE': 45, #North East
    'E': 90, #East
    'SE': 135, #South East 
    'S': 180, #South
    'SW': 225, #South West 
    'W': 270, #West 
    'NW': 315 #North West
}

df['Arah Angin Terbanyak (°)'] = df['Arah Angin Terbanyak (°)'].map(mapping)

# Fill Missing Values using RF Imputer

In [21]:
import numpy as np
import pandas as pd

# STEP 1 — Bersihkan semua invalid values di seluruh kolom
invalid_values = ["-", " - ", "–", "N/A", "n/a", "", " "]
df = df.replace(invalid_values, np.nan)

# STEP 2 — Pisahkan kolom numerik & kategori
numeric_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()

# Tambahkan kolom object yang isinya angka ke numeric_cols
object_cols = df.select_dtypes(include=['object']).columns.tolist()

for col in object_cols:
    if col != 'Tanggal':  # skip tanggal
        # coba convert untuk cek apakah bisa jadi numeric
        temp = pd.to_numeric(df[col], errors="coerce")
        if temp.notnull().sum() > 0:   # kalau ada angka -> anggap numeric
            numeric_cols.append(col)

# Pastikan unique
numeric_cols = list(set(numeric_cols))

# Kolom kategori = semua object kecuali numeric + tanggal
cat_cols = [c for c in object_cols if c not in numeric_cols and c != 'Tanggal']

# STEP 3 — Convert kolom numerik object -> numeric
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# STEP 4 — Impute numerik: median
for col in numeric_cols:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)

# STEP 5 — Impute kategori: mode
for col in cat_cols:
    mode_series = df[col].mode()
    if len(mode_series) > 0:       # antisipasi kalau kolom kosong
        df[col] = df[col].fillna(mode_series[0])

print("Imputasi seluruh kolom selesai!")

# STEP 6 — Simpan hasilnya
df.to_excel("Ready_To_Use_Updated.xlsx", index=False)
print("File berhasil disimpan sebagai Ready_To_Use_Updated.xlsx")


C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\1015568466.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(invalid_values, np.nan)


Imputasi seluruh kolom selesai!
File berhasil disimpan sebagai Ready_To_Use_Updated.xlsx


In [22]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

df = pd.read_excel("Updated_Data_Historis_2015_2025.xlsx", parse_dates=["Tanggal"])
df = df.sort_values("Tanggal").reset_index(drop=True)

invalid_values = ["-", " - ", "–", "—", "N/A", "n/a", "", " ","8888", 8888]
df = df.replace(invalid_values, np.nan)

col = "Arah Angin Terbanyak (°)"

df[col] = (
    df[col].astype(str)
            .str.strip()
            .str.upper()
            .str.replace("–", "-", regex=False)
            .str.replace("—", "-", regex=False)
)

mapping = {
    "C": 0, "N": 0, "NE": 45, "E": 90, "SE": 135,
    "S": 180, "SW": 225, "W": 270, "NW": 315
}

df[col] = df[col].map(mapping)

for column in df.columns:
    if column != "Tanggal":
        df[column] = pd.to_numeric(df[column], errors="ignore")

target = "Curah Hujan (mm)"

# Pastikan target numeric
df[target] = pd.to_numeric(df[target], errors="coerce")

# Pisahkan data train & prediksi (missing only)
train_df = df[df[target].notna()]
predict_df = df[df[target].isna()]

# Fitur lain = semua kolom numeric kecuali target
feature_cols = [col for col in df.columns 
                if col not in ["Tanggal", target] 
                and df[col].dtype != "object"]

X_train = train_df[feature_cols]
y_train = train_df[target]

X_pred = predict_df[feature_cols]

# Train Random Forest
rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)

# Prediksi curah hujan yang hilang
df.loc[df[target].isna(), target] = rf.predict(X_pred)

print("RF imputer untuk Curah Hujan selesai!")

df.to_excel("Ready_To_Use_Updated_RF.xlsx", index=False)
print("File disimpan sebagai Ready_To_Use_Updated_RF.xlsx")
 

C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\2445682571.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(invalid_values, np.nan)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\2445682571.py:30: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[column] = pd.to_numeric(df[column], errors="ignore")


RF imputer untuk Curah Hujan selesai!
File disimpan sebagai Ready_To_Use_Updated_RF.xlsx


In [23]:
data1 = pd.read_excel('Ready_To_Use_Updated_RF.xlsx')

In [24]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4020 non-null   datetime64[ns]
 1   Temperatur Minimum                  3908 non-null   float64       
 2   Temperatur Maksimum                 3979 non-null   float64       
 3   Temperatur Rata-rata                3997 non-null   float64       
 4   Kelembapan Rata-rata                4005 non-null   float64       
 5   Curah Hujan (mm)                    4020 non-null   float64       
 6   Lamanya Penyinaran Matahari         3937 non-null   float64       
 7   Kecepatan Angin Maksimum            4020 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4020 non-null   int64         
 9   Kecepatan Angin Rata-rata           4020 non-null   int64         
 10  Arah Angin Terbanyak (°)

In [25]:
data1.isna().sum().sort_values(ascending=False)


Temperatur Minimum                    112
Lamanya Penyinaran Matahari            83
Temperatur Maksimum                    41
Temperatur Rata-rata                   23
Kelembapan Rata-rata                   15
Tanggal                                 0
Curah Hujan (mm)                        0
Kecepatan Angin Maksimum                0
Arah Angin Saat Kecepatan Maksimum      0
Kecepatan Angin Rata-rata               0
Arah Angin Terbanyak (°)                0
dtype: int64

In [10]:


# === 1. Load Data (setelah imputasi RF yang tadi) ===

df = pd.read_excel("Ready_To_Use_Updated_RF.xlsx")

# Pastikan Tanggal datetime
df['Tanggal'] = pd.to_datetime(df['Tanggal'])

# Sort berdasarkan tanggal
df = df.sort_values('Tanggal')

# === 2. Tentukan kolom fitur yang akan dibuatkan lag ===
# Semua kolom kecuali Tanggal
feature_cols = df.columns.drop('Tanggal')

# === 3. Buat fungsi untuk generate lag otomatis ===
def create_lag_features(df, cols, max_lag=30):
    df = df.copy()

    for col in cols:
        for lag in range(1, max_lag + 1):
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)

    return df

# === 4. Buat lag features t-1 sampai t-30 ===
df_lagged = create_lag_features(df, feature_cols, max_lag=30)

# === 5. Drop baris awal yang menjadi NaN akibat lag ===
df_lagged = df_lagged.dropna().reset_index(drop=True)

# === 6. Simpan hasilnya ===
df_lagged.to_excel("df_feature_engineered.xlsx", index=False)
print("Shape:", df_lagged.shape)


C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\1992546539.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\1992546539.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\1992546539.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

Shape: (1644, 311)


In [12]:
data = pd.read_excel('df_feature_engineered.xlsx')

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1644 entries, 0 to 1643
Columns: 311 entries, Tanggal to Arah Angin Terbanyak (°)_lag_30
dtypes: datetime64[ns](1), float64(155), int64(155)
memory usage: 3.9 MB


In [28]:
# ======================================================
# 1. IMPORT
# ======================================================
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# ======================================================
# 2. LOAD DATA
# ======================================================
df = pd.read_excel("Updated_Data_Historis_2015_2025.xlsx", parse_dates=["Tanggal"])
df = df.sort_values("Tanggal").reset_index(drop=True)

# ======================================================
# 3. CLEAN INVALID VALUES
# ======================================================
invalid_values = ["-", " - ", "–", "—", "N/A", "n/a", "", " ", "8888", 8888]
df = df.replace(invalid_values, np.nan)

# ======================================================
# 4. CLEAN ARAH ANGIN KE DERJAT
# ======================================================
col = "Arah Angin Terbanyak (°)"
df[col] = (
    df[col].astype(str)
            .str.strip()
            .str.upper()
            .str.replace("–", "-", regex=False)
            .str.replace("—", "-", regex=False)
)

mapping = {
    "C": 0, "N": 0, "NE": 45, "E": 90, "SE": 135,
    "S": 180, "SW": 225, "W": 270, "NW": 315
}
df[col] = df[col].map(mapping)

# ======================================================
# 5. KONVERSI SEMUA YG BISA JADI NUMERIC
# ======================================================
for column in df.columns:
    if column != "Tanggal":
        df[column] = pd.to_numeric(df[column], errors="ignore")

# ======================================================
# 6. PISAHKAN NUMERIC & CATEGORICAL
# ======================================================
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

# ======================================================
# 7. IMPUTASI NUMERIC MENGGUNAKAN RANDOM FOREST
# ======================================================
def rf_impute_numeric(df, target_col):
    """Imputasi satu kolom numerik dengan Random Forest"""

    temp = df.copy()

    # data yg lengkap
    train = temp[temp[target_col].notna()]
    # data yang NA
    pred = temp[temp[target_col].isna()]

    if pred.empty:
        return df  # tidak ada NA, skip

    feature_cols = [col for col in numeric_cols if col != target_col]

    X_train = train[feature_cols]
    y_train = train[target_col]

    X_pred = pred[feature_cols]

    rf = RandomForestRegressor(n_estimators=300, random_state=42)
    rf.fit(X_train, y_train)

    temp.loc[temp[target_col].isna(), target_col] = rf.predict(X_pred)

    return temp


print("⏳ Imputasi numeric dengan Random Forest...")
for col in numeric_cols:
    df = rf_impute_numeric(df, col)
print("✅ Imputasi semua kolom numerik selesai!\n")


# ======================================================
# 8. IMPUTASI CATEGORICAL MENGGUNAKAN RANDOM FOREST
# ======================================================
def rf_impute_category(df, target_col):
    """Imputasi satu kolom kategori dengan Random Forest Classifier"""

    temp = df.copy()

    train = temp[temp[target_col].notna()]
    pred = temp[temp[target_col].isna()]

    if pred.empty:
        return df

    # gunakan fitur numerik saja
    feature_cols = numeric_cols

    X_train = train[feature_cols]
    y_train = train[target_col]

    X_pred = pred[feature_cols]

    # label encoding manual
    classes = list(y_train.unique())
    mapping = {c:i for i, c in enumerate(classes)}
    reverse_mapping = {i:c for c, i in mapping.items()}

    y_train_enc = y_train.map(mapping)

    rf = RandomForestClassifier(n_estimators=300, random_state=42)
    rf.fit(X_train, y_train_enc)

    y_pred_enc = rf.predict(X_pred)

    temp.loc[temp[target_col].isna(), target_col] = [reverse_mapping[i] for i in y_pred_enc]

    return temp


print("⏳ Imputasi categorical dengan Random Forest Classifier...")
for col in cat_cols:
    df = rf_impute_category(df, col)
print("✅ Imputasi categorical selesai!")


# ======================================================
# 9. CEK APakah sudah 0 NA
# ======================================================
print("\nCek total NA setelah semua imputasi:")
print(df.isna().sum().sort_values(ascending=False))


# ======================================================
# 10. SIMPAN DATA BERSIH (TANPA NA)
# ======================================================
df.to_excel("RF_Imputed_No_NA.xlsx", index=False)
print("\n✔ File berhasil disimpan: RF_Imputed_No_NA.xlsx")


# ======================================================
# 11. FEATURE ENGINEERING — LAG FEATURES t-1 s/d t-30
# ======================================================
def create_lag_features(df, cols, max_lag=30):
    df = df.copy()
    for col in cols:
        for lag in range(1, max_lag + 1):
            df[f"{col}_lag_{lag}"] = df[col].shift(lag)
    return df


feature_cols = df.columns.drop("Tanggal")

df_lagged = create_lag_features(df, feature_cols, max_lag=30)

print("\nNA setelah lag (wajar karena lag):")
print(df_lagged.isna().sum().sort_values(ascending=False).head())


# ======================================================
# 12. DROP NA yg muncul karena proses LAG
# ======================================================
df_lagged = df_lagged.dropna().reset_index(drop=True)

print("\nCek NA setelah drop lag:")
print(df_lagged.isna().sum().sum())  # harus 0

df_lagged.to_excel("df_feature_engineered_RF2.xlsx", index=False)
print("\n✔ File final disimpan: df_feature_engineered_RF.xlsx")


C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\2346701690.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(invalid_values, np.nan)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\2346701690.py:43: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[column] = pd.to_numeric(df[column], errors="ignore")


⏳ Imputasi numeric dengan Random Forest...
✅ Imputasi semua kolom numerik selesai!

⏳ Imputasi categorical dengan Random Forest Classifier...
✅ Imputasi categorical selesai!

Cek total NA setelah semua imputasi:
Tanggal                               0
Temperatur Minimum                    0
Temperatur Maksimum                   0
Temperatur Rata-rata                  0
Kelembapan Rata-rata                  0
Curah Hujan (mm)                      0
Lamanya Penyinaran Matahari           0
Kecepatan Angin Maksimum              0
Arah Angin Saat Kecepatan Maksimum    0
Kecepatan Angin Rata-rata             0
Arah Angin Terbanyak (°)              0
dtype: int64

✔ File berhasil disimpan: RF_Imputed_No_NA.xlsx

NA setelah lag (wajar karena lag):
Arah Angin Terbanyak (°)_lag_30              30
Kecepatan Angin Rata-rata_lag_30             30
Temperatur Minimum_lag_30                    30
Arah Angin Saat Kecepatan Maksimum_lag_30    30
Temperatur Rata-rata_lag_30                  30
dtype: int

C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\2346701690.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_lag_{lag}"] = df[col].shift(lag)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\2346701690.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_lag_{lag}"] = df[col].shift(lag)
C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_20380\2346701690.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor


✔ File final disimpan: df_feature_engineered_RF.xlsx
